# Glass Formation Prediction based *Ward 2016*

This notebook uses the method from [Ward 2016](https://www.nature.com/articles/npjcompumats201628) to predict glass formation of compounds. Specifically, it uses the set of composition based attributes described in the paper to train a machine learning model.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import matminer

from pymatgen import Composition
from matminer.descriptors import composition_features as cp
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.model_selection import cross_val_score, KFold

%matplotlib inline

## Load training data and compute descriptors
The training data consists of 6836 compounds classified by whether they have amorphous (AM), partially amorphous (AC), or crystalline (CR) structures.

In [2]:
training_set = pd.read_csv("datasets/glass.data", sep=' ')
training_set

,comp,"gfa{AM,AC,CR}"
0,Ag20Al25La55,AM
1,Ag15Al10Mg75,AM
2,Ag25Al10Mg65,AM
3,Ag25Al20Mg55,AM
4,Ag35Al10Mg55,AM
5,Ag35Al20Mg45,AM
6,Ag45Al20Mg35,AM
7,Ag10Ce6Cu84,AM
8,Ag10Ce10Cu80,AM
9,Ag15Ce6Cu79,AM


## Create PyMatGen Composition objects

In [3]:
comp_objects = [Composition(comp) for comp in training_set["comp"]]
comp_objects
training_set = training_set.assign(comp_obj=comp_objects)
training_set

,comp,"gfa{AM,AC,CR}",comp_obj
0,Ag20Al25La55,AM,"(La, Al, Ag)"
1,Ag15Al10Mg75,AM,"(Mg, Al, Ag)"
2,Ag25Al10Mg65,AM,"(Mg, Al, Ag)"
3,Ag25Al20Mg55,AM,"(Mg, Al, Ag)"
4,Ag35Al10Mg55,AM,"(Mg, Al, Ag)"
5,Ag35Al20Mg45,AM,"(Mg, Al, Ag)"
6,Ag45Al20Mg35,AM,"(Mg, Al, Ag)"
7,Ag10Ce6Cu84,AM,"(Ce, Cu, Ag)"
8,Ag10Ce10Cu80,AM,"(Ce, Cu, Ag)"
9,Ag15Ce6Cu79,AM,"(Ce, Cu, Ag)"


## Compute descriptors using MatMiner
Here, we compute 145 composition based attributes as described in *Ward 2016*.

In [4]:
%%time

def calc_attributes(training_set):
    not_attr = list(training_set) #Non-attribute columns
    training_set_updated = cp.StoichAttribute().featurize_dataframe(training_set, col_id="comp_obj")
    training_set_updated = cp.ElemPropertyAttribute().featurize_dataframe(training_set_updated, col_id="comp_obj")
    training_set_updated = cp.ValenceOrbitalAttribute().featurize_dataframe(training_set_updated, col_id="comp_obj")
    training_set_updated = cp.IonicAttribute().featurize_dataframe(training_set_updated, col_id="comp_obj")
    all_cols = list(training_set_updated)
    attr_names = [col for col in all_cols if col not in not_attr]
    return training_set_updated, attr_names

all_descriptors, attr_names = calc_attributes(training_set)
print np.shape(all_descriptors)

(6836, 152)
CPU times: user 20.7 s, sys: 230 ms, total: 20.9 s
Wall time: 20.9 s


## Prediction of glass formation

A random forest classifier model is trained and used to predict glass formation.

In [5]:
gfa = all_descriptors["gfa{AM,AC,CR}"]

model1 = RandomForestClassifier()
model1.fit(all_descriptors[attr_names], gfa)
prediction = model1.predict(all_descriptors[attr_names])

Comparison of actual vs predicted structures. We can see that most of the compounds were classified properly.

In [6]:
pd.crosstab(gfa, prediction, rownames=["actual"], colnames=["predicted"])

predicted,AC,AM,CR
actual,,,
AC,677,28,15
AM,12,4839,17
CR,21,37,1190
